# 여자배구 데이터 분석, 경기 결과 예측 프로젝트

## 라이브러리 import

In [1]:
pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [1]:
import findspark as fs

fs.init()

In [2]:
# spark 생성

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

## 데이터 전처리 - 22/03/12

In [3]:
data = spark.read.csv("hdfs://localhost:9000/data/player.csv", header="true", inferSchema="true")

In [4]:
data.show()

+---+------+------------+---+------+-----+--------+------+-------+-------+--------+-------+------+-------+-------+-------+
|_c0|  name|        team|pos| score|error|setcount|at_try|at_succ|sv_succ|set_succ|bl_succ|rs_try|rs_corr|rs_fail|dg_succ|
+---+------+------------+---+------+-----+--------+------+-------+-------+--------+-------+------+-------+-------+-------+
|  1|임명옥|한국도로공사| Li|1475.0|    3|     117|     1|      0|      0|     123|      0|   677|    390|     16|    690|
|  2|신연경| IBK기업은행| Li|1158.6|    3|     112|     1|      0|      0|      67|      0|   525|    229|     22|    591|
|  3|김연견|    현대건설| Li|1099.0|    4|     113|     0|      0|      0|      89|      0|   477|    213|     26|    560|
|  4|오지영|    GS칼텍스| Li| 880.6|    2|      94|     1|      0|      0|      48|      0|   541|    249|     36|    375|
|  5|문슬기|페퍼저축은행| Li| 841.8|    2|      99|     1|      0|      0|      69|      0|   568|    224|     42|    307|
|  6|  노란| KGC인삼공사| Li| 819.8|    5|      83|     3|  

In [15]:
data.createOrReplaceTempView("player")

In [23]:
player = spark.sql("select _c0,name,team,pos,score from player")
player.show()

+---+------+------------+---+------+
|_c0|  name|        team|pos| score|
+---+------+------------+---+------+
|  1|임명옥|한국도로공사| Li|1475.0|
|  2|신연경| IBK기업은행| Li|1158.6|
|  3|김연견|    현대건설| Li|1099.0|
|  4|오지영|    GS칼텍스| Li| 880.6|
|  5|문슬기|페퍼저축은행| Li| 841.8|
|  6|  노란| KGC인삼공사| Li| 819.8|
|  7|김해란|    흥국생명| Li| 596.4|
|  8|도수빈|    흥국생명| Li| 589.2|
|  9|김세인|페퍼저축은행| Li| 334.4|
| 10|채선아| KGC인삼공사| Li| 331.2|
| 11|한수진|    GS칼텍스| Li| 222.6|
| 12|한다혜|    GS칼텍스| Li|  86.2|
| 13|김수빈| IBK기업은행| Li|  83.6|
| 14|이영주|    현대건설| Li|  62.2|
| 15|김주하|    현대건설| Li|  59.8|
| 16|박상미|    흥국생명| Li|  42.2|
| 17|이은지|페퍼저축은행| Li|   7.8|
| 18|김주희|    GS칼텍스| Li|   3.0|
| 19|박혜미|한국도로공사| Li|   3.0|
| 20|차유정|    GS칼텍스| Li|   3.0|
+---+------+------------+---+------+
only showing top 20 rows



In [37]:
spark.sql("select distinct team from player").show()

+------------+
|        team|
+------------+
|    흥국생명|
|한국도로공사|
|    GS칼텍스|
|    현대건설|
|페퍼저축은행|
| KGC인삼공사|
| IBK기업은행|
+------------+



In [46]:
hgun = spark.sql("""
          select *
          from player
          where team like '현대건설'
          """)
hgun.show(5)

+---+------+--------+---+------+-----+--------+------+-------+-------+--------+-------+------+-------+-------+-------+
|_c0|  name|    team|pos| score|error|setcount|at_try|at_succ|sv_succ|set_succ|bl_succ|rs_try|rs_corr|rs_fail|dg_succ|
+---+------+--------+---+------+-----+--------+------+-------+-------+--------+-------+------+-------+-------+-------+
|  3|김연견|현대건설| Li|1099.0|    4|     113|     0|      0|      0|      89|      0|   477|    213|     26|    560|
| 14|이영주|현대건설| Li|  62.2|    2|      21|     1|      0|      0|       4|      0|    15|      6|      2|     28|
| 15|김주하|현대건설| Li|  59.8|    0|       5|     0|      0|      0|       5|      0|    24|     11|      3|     34|
| 21|한미르|현대건설| Li|   1.4|    0|       1|     0|      0|      0|       0|      0|     0|      0|      0|      0|
| 33|황민경|현대건설|  L|1815.0|   69|     112|   536|    171|     21|      28|     14|   527|    186|     22|    428|
+---+------+--------+---+------+-----+--------+------+-------+-------+--------+----

In [41]:
dogong = spark.sql("""
          select *
          from player
          where team like '한국도로공사'
          """)
dogong.show(5)

+---+------+------------+---+------+-----+--------+------+-------+-------+--------+-------+------+-------+-------+-------+
|_c0|  name|        team|pos| score|error|setcount|at_try|at_succ|sv_succ|set_succ|bl_succ|rs_try|rs_corr|rs_fail|dg_succ|
+---+------+------------+---+------+-----+--------+------+-------+-------+--------+-------+------+-------+-------+-------+
|  1|임명옥|한국도로공사| Li|1475.0|    3|     117|     1|      0|      0|     123|      0|   677|    390|     16|    690|
| 19|박혜미|한국도로공사| Li|   3.0|    0|       3|     0|      0|      0|       0|      0|     0|      0|      0|      0|
| 22|김정아|한국도로공사| Li|   1.0|    0|       1|     0|      0|      0|       0|      0|     0|      0|      0|      0|
| 30|박정아|한국도로공사|  L|2094.0|  113|     115|  1153|    393|     13|      13|     34|   141|     26|     16|    256|
| 37|전새얀|한국도로공사|  L|1500.0|   49|     115|   510|    179|      7|      14|     39|   556|    163|     28|    192|
+---+------+------------+---+------+-----+--------+------+---

In [45]:
gscal = spark.sql("""
          select *
          from player
          where team like 'GS칼텍스'
          """)
gscal.show(5)

+---+------+--------+---+-----+-----+--------+------+-------+-------+--------+-------+------+-------+-------+-------+
|_c0|  name|    team|pos|score|error|setcount|at_try|at_succ|sv_succ|set_succ|bl_succ|rs_try|rs_corr|rs_fail|dg_succ|
+---+------+--------+---+-----+-----+--------+------+-------+-------+--------+-------+------+-------+-------+-------+
|  4|오지영|GS칼텍스| Li|880.6|    2|      94|     1|      0|      0|      48|      0|   541|    249|     36|    375|
| 11|한수진|GS칼텍스| Li|222.6|    0|      59|     0|      0|      0|      15|      0|     0|      0|      0|    133|
| 12|한다혜|GS칼텍스| Li| 86.2|    0|      15|     0|      0|      0|       4|      0|    56|     29|      0|     24|
| 18|김주희|GS칼텍스| Li|  3.0|    0|       3|     0|      0|      0|       0|      0|     0|      0|      0|      0|
| 20|차유정|GS칼텍스| Li|  3.0|    0|       3|     0|      0|      0|       0|      0|     0|      0|      0|      0|
+---+------+--------+---+-----+-----+--------+------+-------+-------+--------+-------+

In [42]:
insam = spark.sql("""
          select *
          from player
          where team like 'KGC인삼공사'
          """)
insam.show(5)

+---+------+-----------+---+------+-----+--------+------+-------+-------+--------+-------+------+-------+-------+-------+
|_c0|  name|       team|pos| score|error|setcount|at_try|at_succ|sv_succ|set_succ|bl_succ|rs_try|rs_corr|rs_fail|dg_succ|
+---+------+-----------+---+------+-----+--------+------+-------+-------+--------+-------+------+-------+-------+-------+
|  6|  노란|KGC인삼공사| Li| 819.8|    5|      83|     3|      0|      0|      54|      0|   357|    157|     12|    416|
| 10|채선아|KGC인삼공사| Li| 331.2|    1|      37|     0|      0|      0|      25|      0|   183|     82|     10|    140|
| 23|서유경|KGC인삼공사| Li|   1.0|    0|       1|     0|      0|      0|       0|      0|     0|      0|      0|      0|
| 26|이소영|KGC인삼공사|  L|2311.4|  102|     105|   873|    310|     22|      32|     31|   616|    267|     23|    391|
| 38|박혜민|KGC인삼공사|  L|1440.6|   68|      85|   486|    173|     11|      13|     16|   528|    182|     29|    251|
+---+------+-----------+---+------+-----+--------+------+-

In [43]:
ibk = spark.sql("""
          select *
          from player
          where team like 'IBK기업은행'
          """)
ibk.show(5)

+---+------+-----------+---+------+-----+--------+------+-------+-------+--------+-------+------+-------+-------+-------+
|_c0|  name|       team|pos| score|error|setcount|at_try|at_succ|sv_succ|set_succ|bl_succ|rs_try|rs_corr|rs_fail|dg_succ|
+---+------+-----------+---+------+-----+--------+------+-------+-------+--------+-------+------+-------+-------+-------+
|  2|신연경|IBK기업은행| Li|1158.6|    3|     112|     1|      0|      0|      67|      0|   525|    229|     22|    591|
| 13|김수빈|IBK기업은행| Li|  83.6|    0|      26|     0|      0|      0|       2|      0|    15|      5|      1|     39|
| 24|양유경|IBK기업은행| Li|   1.0|    0|       1|     0|      0|      0|       0|      0|     0|      0|      0|      0|
| 28|표승주|IBK기업은행|  L|2175.4|   57|     102|   775|    285|     17|      24|     28|   755|    231|     30|    302|
| 35|김주향|IBK기업은행|  L|1704.4|   76|      91|   721|    238|      8|      10|     20|   521|    163|     23|    267|
+---+------+-----------+---+------+-----+--------+------+--

In [44]:
hguk = spark.sql("""
          select *
          from player
          where team like '흥국생명'
          """)
hguk.show(5)

+---+------+--------+---+------+-----+--------+------+-------+-------+--------+-------+------+-------+-------+-------+
|_c0|  name|    team|pos| score|error|setcount|at_try|at_succ|sv_succ|set_succ|bl_succ|rs_try|rs_corr|rs_fail|dg_succ|
+---+------+--------+---+------+-----+--------+------+-------+-------+--------+-------+------+-------+-------+-------+
|  7|김해란|흥국생명| Li| 596.4|    2|      58|     0|      0|      0|      59|      0|   211|     82|     16|    335|
|  8|도수빈|흥국생명| Li| 589.2|    3|      60|     1|      0|      0|      38|      0|   271|    116|     12|    294|
| 16|박상미|흥국생명| Li|  42.2|    0|      16|     0|      0|      0|       0|      0|     3|      1|      1|     23|
| 25|현무린|흥국생명| Li|   1.0|    0|       1|     0|      0|      0|       0|      0|     0|      0|      0|      0|
| 27|김미연|흥국생명|  L|2238.2|  102|     111|   751|    250|     27|      35|     21|   790|    251|     49|    406|
+---+------+--------+---+------+-----+--------+------+-------+-------+--------+----

In [52]:
pepper = spark.sql("""
          select *
          from player
          where team like '페퍼저축은행'
          """)
pepper.show(5)

+---+------+------------+---+------+-----+--------+------+-------+-------+--------+-------+------+-------+-------+-------+
|_c0|  name|        team|pos| score|error|setcount|at_try|at_succ|sv_succ|set_succ|bl_succ|rs_try|rs_corr|rs_fail|dg_succ|
+---+------+------------+---+------+-----+--------+------+-------+-------+--------+-------+------+-------+-------+-------+
|  5|문슬기|페퍼저축은행| Li| 841.8|    2|      99|     1|      0|      0|      69|      0|   568|    224|     42|    307|
|  9|김세인|페퍼저축은행| Li| 334.4|    0|      84|     0|      0|      0|      19|      0|    36|     11|      0|    185|
| 17|이은지|페퍼저축은행| Li|   7.8|    0|       4|     0|      0|      0|       0|      0|     0|      0|      0|      2|
| 32|이한비|페퍼저축은행|  L|1930.4|   91|     105|   791|    238|     11|      21|     13|   695|    237|     33|    334|
| 34|박경현|페퍼저축은행|  L|1714.8|   94|      96|   610|    217|     15|      22|     10|   741|    205|     41|    275|
+---+------+------------+---+------+-----+--------+------+---

In [56]:
# 공성, 공효(보류), 리효, 세트당 서브/세트/블로킹/디그 계산

player_rate = spark.sql("""
          select name,team,pos,score,error,setcount,round(at_succ/at_try,2) as at_srate,
              round((rs_corr-rs_fail)/rs_try,2) as rs_rate, round(sv_succ/setcount,2) as sv, 
              round(set_succ/setcount,2) as set,round(bl_succ/setcount,2) as bl,round(dg_succ/setcount,2) as dg
          from player
          """)

player_rate.show()

+------+------------+---+------+-----+--------+--------+-------+---+----+---+----+
|  name|        team|pos| score|error|setcount|at_srate|rs_rate| sv| set| bl|  dg|
+------+------------+---+------+-----+--------+--------+-------+---+----+---+----+
|임명옥|한국도로공사| Li|1475.0|    3|     117|     0.0|   0.55|0.0|1.05|0.0| 5.9|
|신연경| IBK기업은행| Li|1158.6|    3|     112|     0.0|   0.39|0.0| 0.6|0.0|5.28|
|김연견|    현대건설| Li|1099.0|    4|     113|    null|   0.39|0.0|0.79|0.0|4.96|
|오지영|    GS칼텍스| Li| 880.6|    2|      94|     0.0|   0.39|0.0|0.51|0.0|3.99|
|문슬기|페퍼저축은행| Li| 841.8|    2|      99|     0.0|   0.32|0.0| 0.7|0.0| 3.1|
|  노란| KGC인삼공사| Li| 819.8|    5|      83|     0.0|   0.41|0.0|0.65|0.0|5.01|
|김해란|    흥국생명| Li| 596.4|    2|      58|    null|   0.31|0.0|1.02|0.0|5.78|
|도수빈|    흥국생명| Li| 589.2|    3|      60|     0.0|   0.38|0.0|0.63|0.0| 4.9|
|김세인|페퍼저축은행| Li| 334.4|    0|      84|    null|   0.31|0.0|0.23|0.0| 2.2|
|채선아| KGC인삼공사| Li| 331.2|    1|      37|    null|   0.39|0.0|0.68|0.0|3

In [58]:
player_rate.createOrReplaceTempView("player_rate")

In [59]:
hgun_rate = spark.sql("""
          select *
          from player_rate
          where team like '현대건설'
          """)
hgun_rate.show(5)

+------+--------+---+------+-----+--------+--------+-------+----+----+----+----+
|  name|    team|pos| score|error|setcount|at_srate|rs_rate|  sv| set|  bl|  dg|
+------+--------+---+------+-----+--------+--------+-------+----+----+----+----+
|김연견|현대건설| Li|1099.0|    4|     113|    null|   0.39| 0.0|0.79| 0.0|4.96|
|이영주|현대건설| Li|  62.2|    2|      21|     0.0|   0.27| 0.0|0.19| 0.0|1.33|
|김주하|현대건설| Li|  59.8|    0|       5|    null|   0.33| 0.0| 1.0| 0.0| 6.8|
|한미르|현대건설| Li|   1.4|    0|       1|    null|   null| 0.0| 0.0| 0.0| 0.0|
|황민경|현대건설|  L|1815.0|   69|     112|    0.32|   0.31|0.19|0.25|0.13|3.82|
+------+--------+---+------+-----+--------+--------+-------+----+----+----+----+
only showing top 5 rows



In [60]:
dogong_rate = spark.sql("""
          select *
          from player_rate
          where team like '한국도로공사'
          """)
dogong_rate.show(5)

+------+------------+---+------+-----+--------+--------+-------+----+----+----+----+
|  name|        team|pos| score|error|setcount|at_srate|rs_rate|  sv| set|  bl|  dg|
+------+------------+---+------+-----+--------+--------+-------+----+----+----+----+
|임명옥|한국도로공사| Li|1475.0|    3|     117|     0.0|   0.55| 0.0|1.05| 0.0| 5.9|
|박혜미|한국도로공사| Li|   3.0|    0|       3|    null|   null| 0.0| 0.0| 0.0| 0.0|
|김정아|한국도로공사| Li|   1.0|    0|       1|    null|   null| 0.0| 0.0| 0.0| 0.0|
|박정아|한국도로공사|  L|2094.0|  113|     115|    0.34|   0.07|0.11|0.11| 0.3|2.23|
|전새얀|한국도로공사|  L|1500.0|   49|     115|    0.35|   0.24|0.06|0.12|0.34|1.67|
+------+------------+---+------+-----+--------+--------+-------+----+----+----+----+
only showing top 5 rows



In [61]:
gscal_rate = spark.sql("""
          select *
          from player_rate
          where team like 'GS칼텍스'
          """)
gscal_rate.show(5)

+------+--------+---+-----+-----+--------+--------+-------+---+----+---+----+
|  name|    team|pos|score|error|setcount|at_srate|rs_rate| sv| set| bl|  dg|
+------+--------+---+-----+-----+--------+--------+-------+---+----+---+----+
|오지영|GS칼텍스| Li|880.6|    2|      94|     0.0|   0.39|0.0|0.51|0.0|3.99|
|한수진|GS칼텍스| Li|222.6|    0|      59|    null|   null|0.0|0.25|0.0|2.25|
|한다혜|GS칼텍스| Li| 86.2|    0|      15|    null|   0.52|0.0|0.27|0.0| 1.6|
|김주희|GS칼텍스| Li|  3.0|    0|       3|    null|   null|0.0| 0.0|0.0| 0.0|
|차유정|GS칼텍스| Li|  3.0|    0|       3|    null|   null|0.0| 0.0|0.0| 0.0|
+------+--------+---+-----+-----+--------+--------+-------+---+----+---+----+
only showing top 5 rows



In [62]:
insam_rate = spark.sql("""
          select *
          from player_rate
          where team like 'KGC인삼공사'
          """)
insam_rate.show(5)

+------+-----------+---+------+-----+--------+--------+-------+----+----+----+----+
|  name|       team|pos| score|error|setcount|at_srate|rs_rate|  sv| set|  bl|  dg|
+------+-----------+---+------+-----+--------+--------+-------+----+----+----+----+
|  노란|KGC인삼공사| Li| 819.8|    5|      83|     0.0|   0.41| 0.0|0.65| 0.0|5.01|
|채선아|KGC인삼공사| Li| 331.2|    1|      37|    null|   0.39| 0.0|0.68| 0.0|3.78|
|서유경|KGC인삼공사| Li|   1.0|    0|       1|    null|   null| 0.0| 0.0| 0.0| 0.0|
|이소영|KGC인삼공사|  L|2311.4|  102|     105|    0.36|    0.4|0.21| 0.3| 0.3|3.72|
|박혜민|KGC인삼공사|  L|1440.6|   68|      85|    0.36|   0.29|0.13|0.15|0.19|2.95|
+------+-----------+---+------+-----+--------+--------+-------+----+----+----+----+
only showing top 5 rows



In [63]:
ibk_rate = spark.sql("""
          select *
          from player_rate
          where team like 'IBK기업은행'
          """)
ibk_rate.show(5)

+------+-----------+---+------+-----+--------+--------+-------+----+----+----+----+
|  name|       team|pos| score|error|setcount|at_srate|rs_rate|  sv| set|  bl|  dg|
+------+-----------+---+------+-----+--------+--------+-------+----+----+----+----+
|신연경|IBK기업은행| Li|1158.6|    3|     112|     0.0|   0.39| 0.0| 0.6| 0.0|5.28|
|김수빈|IBK기업은행| Li|  83.6|    0|      26|    null|   0.27| 0.0|0.08| 0.0| 1.5|
|양유경|IBK기업은행| Li|   1.0|    0|       1|    null|   null| 0.0| 0.0| 0.0| 0.0|
|표승주|IBK기업은행|  L|2175.4|   57|     102|    0.37|   0.27|0.17|0.24|0.27|2.96|
|김주향|IBK기업은행|  L|1704.4|   76|      91|    0.33|   0.27|0.09|0.11|0.22|2.93|
+------+-----------+---+------+-----+--------+--------+-------+----+----+----+----+
only showing top 5 rows



In [64]:
hguk_rate = spark.sql("""
          select *
          from player_rate
          where team like '흥국생명'
          """)
hguk_rate.show(5)

+------+--------+---+------+-----+--------+--------+-------+----+----+----+----+
|  name|    team|pos| score|error|setcount|at_srate|rs_rate|  sv| set|  bl|  dg|
+------+--------+---+------+-----+--------+--------+-------+----+----+----+----+
|김해란|흥국생명| Li| 596.4|    2|      58|    null|   0.31| 0.0|1.02| 0.0|5.78|
|도수빈|흥국생명| Li| 589.2|    3|      60|     0.0|   0.38| 0.0|0.63| 0.0| 4.9|
|박상미|흥국생명| Li|  42.2|    0|      16|    null|    0.0| 0.0| 0.0| 0.0|1.44|
|현무린|흥국생명| Li|   1.0|    0|       1|    null|   null| 0.0| 0.0| 0.0| 0.0|
|김미연|흥국생명|  L|2238.2|  102|     111|    0.33|   0.26|0.24|0.32|0.19|3.66|
+------+--------+---+------+-----+--------+--------+-------+----+----+----+----+
only showing top 5 rows



In [65]:
pepper_rate = spark.sql("""
          select *
          from player_rate
          where team like '페퍼저축은행'
          """)
pepper_rate.show(5)

+------+------------+---+------+-----+--------+--------+-------+----+----+----+----+
|  name|        team|pos| score|error|setcount|at_srate|rs_rate|  sv| set|  bl|  dg|
+------+------------+---+------+-----+--------+--------+-------+----+----+----+----+
|문슬기|페퍼저축은행| Li| 841.8|    2|      99|     0.0|   0.32| 0.0| 0.7| 0.0| 3.1|
|김세인|페퍼저축은행| Li| 334.4|    0|      84|    null|   0.31| 0.0|0.23| 0.0| 2.2|
|이은지|페퍼저축은행| Li|   7.8|    0|       4|    null|   null| 0.0| 0.0| 0.0| 0.5|
|이한비|페퍼저축은행|  L|1930.4|   91|     105|     0.3|   0.29| 0.1| 0.2|0.12|3.18|
|박경현|페퍼저축은행|  L|1714.8|   94|      96|    0.36|   0.22|0.16|0.23| 0.1|2.86|
+------+------------+---+------+-----+--------+--------+-------+----+----+----+----+
only showing top 5 rows



In [86]:
# 선수 데이터 저장 - 하둡

player_rate.write.format('csv').save('hdfs://localhost:9000/data/player_r.csv',header='true')

In [88]:
data = spark.read.csv("hdfs://localhost:9000/data/player_r.csv", header='true', inferSchema='true')
data.show(5)

+------+------------+---+------+-----+--------+--------+-------+---+----+---+----+
|  name|        team|pos| score|error|setcount|at_srate|rs_rate| sv| set| bl|  dg|
+------+------------+---+------+-----+--------+--------+-------+---+----+---+----+
|임명옥|한국도로공사| Li|1475.0|    3|     117|     0.0|   0.55|0.0|1.05|0.0| 5.9|
|신연경| IBK기업은행| Li|1158.6|    3|     112|     0.0|   0.39|0.0| 0.6|0.0|5.28|
|김연견|    현대건설| Li|1099.0|    4|     113|    null|   0.39|0.0|0.79|0.0|4.96|
|오지영|    GS칼텍스| Li| 880.6|    2|      94|     0.0|   0.39|0.0|0.51|0.0|3.99|
|문슬기|페퍼저축은행| Li| 841.8|    2|      99|     0.0|   0.32|0.0| 0.7|0.0| 3.1|
+------+------------+---+------+-----+--------+--------+-------+---+----+---+----+
only showing top 5 rows



In [81]:
# 구단별 데이터 저장 - 하둡

hgun_rate.write.format('csv').save('hdfs://localhost:9000/data/hgun_r.csv',header='true')
dogong_rate.write.format('csv').save('hdfs://localhost:9000/data/dogong_r.csv',header='true')
gscal_rate.write.format('csv').save('hdfs://localhost:9000/data/gscal_r.csv',header='true')
insam_rate.write.format('csv').save('hdfs://localhost:9000/data/insam_r.csv',header='true')
ibk_rate.write.format('csv').save('hdfs://localhost:9000/data/ibk_r.csv',header='true')
hguk_rate.write.format('csv').save('hdfs://localhost:9000/data/hguk_r.csv',header='true')
pepper_rate.write.format('csv').save('hdfs://localhost:9000/data/pepper_r.csv',header='true')

In [85]:
# 데이터 잘 저장됐는지 확인

data = spark.read.csv("hdfs://localhost:9000/data/pepper_r.csv", header='true', inferSchema="true")
data.show(5)

+------+------------+---+------+-----+--------+--------+-------+----+----+----+----+
|  name|        team|pos| score|error|setcount|at_srate|rs_rate|  sv| set|  bl|  dg|
+------+------------+---+------+-----+--------+--------+-------+----+----+----+----+
|문슬기|페퍼저축은행| Li| 841.8|    2|      99|     0.0|   0.32| 0.0| 0.7| 0.0| 3.1|
|김세인|페퍼저축은행| Li| 334.4|    0|      84|    null|   0.31| 0.0|0.23| 0.0| 2.2|
|이은지|페퍼저축은행| Li|   7.8|    0|       4|    null|   null| 0.0| 0.0| 0.0| 0.5|
|이한비|페퍼저축은행|  L|1930.4|   91|     105|     0.3|   0.29| 0.1| 0.2|0.12|3.18|
|박경현|페퍼저축은행|  L|1714.8|   94|      96|    0.36|   0.22|0.16|0.23| 0.1|2.86|
+------+------------+---+------+-----+--------+--------+-------+----+----+----+----+
only showing top 5 rows



## 데이터 모델 생성 - 22/